In [15]:
##
import os
import sys
import warnings
import enum
from enum import Enum
import matplotlib
from pathlib import Path
from typing import Callable

##
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

##
from finrl import config as finrl_config
from finrl.main import check_and_make_directories
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.config import (
    TEST_END_DATE,
    TRAINED_MODEL_DIR,
)
from finrl.meta.preprocessor.preprocessors import data_split
from finrl.plot import backtest_stats, convert_daily_return_to_pyfolio_ts, get_baseline

##
import yfinance as yf

##
from meta.data_processors.yahoofinance import Yahoofinance

In [16]:
def config():
    # #
    warnings.filterwarnings("ignore", category=UserWarning)  # TODO: zipline problem
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    # os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # INFO, WARN are ignored and only ERROR messages will be printed

    # #
    matplotlib.use("Agg")

    # #
    check_and_make_directories(
        [
            finrl_config.DATA_SAVE_DIR,
            finrl_config.TRAINED_MODEL_DIR,
            finrl_config.TENSORBOARD_LOG_DIR,
            finrl_config.RESULTS_DIR,
        ]
    )


config()

In [17]:
##
import numpy as np
import pandas as pd
from pyfolio import timeseries
import tqdm

##
from rl.data import DataTechnicalAnalysis, DataFundamentalAnalysis, DataBase
from common.utils import now_time
from configuration.settings import ProjectDir

In [18]:
def print_info(df: pd.DataFrame):
    ##
    print(f"{df.shape=}")
    print(f"Start date: {df['date'].iloc[0]}")
    print(f"End date: {df['date'].iloc[-1]}")
    print(f"Stocks: {df['tic'].unique().size}")

    ##
    data_grouped_by_date = df.groupby(by=["date"])
    grouped_data_size = data_grouped_by_date.size()
    if grouped_data_size.nunique() == 1:
        print(f"OK: All dates have same row count: {grouped_data_size.unique()}")
    else:
        raise ValueError(f"Different size for each date: {grouped_data_size.unique()}")


def check_dir(_filename: Path):
    dir = _filename.parent
    if os.path.isdir(dir):
        print(f"OK: Dir exist for: {_filename}")
    else:
        raise NotADirectoryError(f"ERROR: Dir does not exist {dir}")


class FileType(Enum):
    JSON = enum.auto
    CSV = enum.auto


def save_dataset(dataframe: pd.DataFrame, type: FileType, _filename: str):
    print(f"Saving data into: {_filename}")
    if type == FileType.JSON:
        dataframe.to_json(_filename)
    elif type == FileType.CSV:
        dataframe.to_csv(_filename)
    else:
        raise ValueError(f"type: FileType must be CSV or JSON.")
    print(f"Data saved to json: {_filename}")

In [19]:
prj_dir = ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing"))
_TRAIN_DATA_START = "2010-01-01"
_TRAIN_DATA_END = "2021-12-31"
_TEST_DATA_START = "2021-01-01"
_TEST_DATA_END = "2021-12-31"
tickers = DOW_30_TICKER

In [7]:
print(tickers.__len__())
print(tickers)
from meta.data_processors.yahoofinance import Yahoofinance

data_processor = Yahoofinance(
    data_source="yahoofinance", start_date=_TRAIN_DATA_START, end_date=_TEST_DATA_END, time_interval="1d"
)

30
['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [8]:
data_processor.download_data(ticker_list=tickers)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[29 30]


In [10]:
print(data_processor.dataframe.shape)
data_processor.dataframe.head()
print_info(data_processor.dataframe)

(88283, 9)
df.shape=(88283, 9)
Start date: 2010-01-04
End date: 2021-12-30
Stocks: 30


ValueError: Different size for each date: [29 30]

In [246]:
# df = data_processor.dataframe

# for i in range(1):
#     date1 = df['date'].unique()[-i]
#     grouped_by_date = df.groupby(by=["date"])
#     grouped_by_date_size = grouped_by_date.size()
#     bad_date = df[df['date'] == date1]
#     print(bad_date.shape)


# sizes = df.groupby('date')['date'].transform('size')
# sizes
# df.drop_duplicates(subset='date', keep='first')

# bad_date2
# print(bad_date['tic'].unique().size)
# print(bad_date2['tic'].unique().size)

# grouped_by_df = df.groupby(by=["date"])
# grouped_by_size = grouped_by_df.size()
# # print(grouped_by_df)
# print(grouped_by_size)
# print(grouped_by_size.unique())
# counts = pd.Series([])
# for i in grouped_by_size.unique():
#     occurences = (grouped_by_size == i).sum()
#     counts = counts.append(pd.Series(occurences))
#     print(f"{i} : {occurences}")

date
2010-01-04    29
2010-01-05    29
2010-01-06    29
2010-01-07    29
2010-01-08    29
              ..
2021-12-23    30
2021-12-27    30
2021-12-28    30
2021-12-29    30
2021-12-30    30
Length: 3020, dtype: int64
[29 30]
29 : 2317
30 : 703


In [61]:
def get_missing_tic(df: pd.DataFrame, date: str) -> list:
    df_date = df[df["date"] == date]
    missing_tics: set = set(tickers).difference(set(df_date["tic"].values))
    missing_tics: list = list(missing_tics)
    return [missing_tics]


def get_previous_value(df: pd.DataFrame, date: str, missing_tic: str):
    df_before_date_binary = df["date"] < date
    df_before_date = df[df_before_date_binary]
    previous_value_of_missing_tic = df_before_date["tic"] == missing_tic[0]
    df_missing_tic_previous_value = df_before_date[previous_value_of_missing_tic]
    if df_missing_tic_previous_value.empty:
        return None
    else:
        return df_missing_tic_previous_value.iloc[-1]


def get_following_value(df: pd.DataFrame, date: str, missing_tic: str):
    df_before_date_binary = df["date"] > date
    df_before_date = df[df_before_date_binary]
    previous_value_of_missing_tic_binary = df_before_date["tic"] == missing_tic[0]
    df_missing_tic_previous_value = df_before_date[previous_value_of_missing_tic_binary]
    if df_missing_tic_previous_value.empty:
        return None
    else:
        return df_missing_tic_previous_value.iloc[0]


def fill_missed_tic_gaps(df: pd.DataFrame) -> pd.DataFrame:
    grouped_by_df = data_processor.dataframe.groupby(by=["date"])
    grouped_by_size = grouped_by_df.size()
    max_size = grouped_by_size.max()
    # print(grouped_by_size.unique())

    for date in grouped_by_size[grouped_by_size < max_size].index:  # for each date where some tics are missed
        ##
        # print(date)
        tics = get_missing_tic(df, date)
        # print(tics)

        for tic in tics:
            filled_value = get_previous_value(df, date, missing_tic=tic)
            filled_value = filled_value if filled_value is not None else get_following_value(df, date, missing_tic=tic)
            if filled_value is None:
                raise ValueError("None value of filling value")
            else:
                filled_value["date"] = date
                df = df.append(filled_value)

    return df


def get_difference_in_timeframe():
    grouped_by_df = df_filled_gaps.groupby(by=["date"])
    grouped_by_size = grouped_by_df.size()
    return grouped_by_size


df_filled_gaps = fill_missed_tic_gaps(data_processor.dataframe)
print(get_difference_in_timeframe().unique())

[30]


In [255]:
# _d = df['date'] <= '2011-01-04'
# prtin(_d == True).sum()
# prtin(_d == False).sum()

80917

In [167]:
s = pd.Series([5, 5, 5, 6, 6])

u = s.unique()
print(u)
c = (s == u[0]).sum()
print(c)
idxs = s.index[s.isin([u[1]])]
print(idxs)

[5 6]
3
Int64Index([3, 4], dtype='int64')


In [ ]:
dataframe = pd.DataFrame()

for tic in tickers:
    temp_df = yf.download(
        tic,
        start=_TRAIN_DATA_START,
        end=_TEST_DATA_END,
        interval="1d",
    )
    temp_df["tic"] = tic
    dataframe = pd.concat([dataframe, temp_df], axis=0, join="outer")
dataframe.reset_index(inplace=True)

In [ ]:
filename = data_processor.get_filename(prj_dir=prj_dir, name="dji30_ta")
check_dir(filename)

# %load_ext line_profiler
# %lprun -f save_dataset save_dataset()

In [ ]:

save_dataset(dataframe=data_processor.dataframe, type=FileType.JSON, filename)


In [ ]:
data_processor.dataframe.shape

In [ ]:
fe = FeatureEngineer(use_technical_indicator=True, use_turbulence=False, user_defined_feature=False)
data_features: pd.DataFrame = fe.preprocess_data(data_processor.dataframe.copy())

In [ ]:
data_features.shape

In [14]:

save_dataset(dataframe=data_processor.dataframe, type=FileType.JSON, filename)


Saving data into: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-14T20-13-55.json
Data saved to json: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-14T20-13-55.json


In [11]:
def add_cov(df: pd.DataFrame) -> pd.DataFrame:
    ##
    # add covariance matrix as states
    df = df.sort_values(["date", "tic"], ignore_index=True)
    df.index = df.date.factorize()[0]

    cov_list = []
    return_list = []

    # look back is one year
    lookback = 252
    for i in tqdm.tqdm(range(lookback, len(df.index.unique()))):
        data_lookback = df.loc[i - lookback : i, :]
        price_lookback = data_lookback.pivot_table(index="date", columns="tic", values="close")
        return_lookback = price_lookback.pct_change().dropna()
        covs = return_lookback.cov().values

        #
        return_list.append(return_lookback)
        cov_list.append(covs)

    df_cov = pd.DataFrame({"date": df.date.unique()[lookback:], "cov_list": cov_list, "return_list": return_list})
    df = df.merge(df_cov, on="date")
    df = df.sort_values(["date", "tic"]).reset_index(drop=True)
    return df


data_features_cov1 = add_cov(data_features.copy())

NameError: name 'data_features' is not defined

In [10]:
print(data_features_cov1.shape)

NameError: name 'data_features_cov1' is not defined

In [17]:
# _df = data_features.copy()

# rolling_window =
# cov_matrix = _df.rolling(window=252).cov()

# print(rolling_window)
# print(_df.shape)

# data_features_cov2 = cov2(data_features.copy())

In [18]:
# print(data_features_cov2.shape)

In [37]:
print_info(data_processor.dataframe)

df.shape=(0, 0)


KeyError: 'date'

In [29]:
data_features_cov1.head()

,date,open,high,low,close,adjusted_close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2011-01-03,11.630000,11.795000,11.601429,11.770357,10.033262,445138400,AAPL,0,0.118248,11.709899,11.326780,62.862180,144.286559,25.488753,11.415357,11.212333,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
1,2011-01-03,55.200001,56.279999,55.180000,55.549999,41.640541,5453300,AMGN,0,0.261720,58.568809,52.495191,50.863117,34.401812,4.927279,54.949000,55.565166,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
2,2011-01-03,43.299999,43.619999,43.110001,43.400002,36.488426,7633300,AXP,0,-0.178628,47.098241,41.260759,51.443736,-26.423899,13.049417,43.883000,42.473000,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
3,2011-01-03,66.150002,66.680000,66.000000,66.400002,53.034523,8072900,BA,0,-0.081580,66.520526,63.197475,51.659856,149.651676,11.924376,64.853001,66.948501,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
4,2011-01-03,94.379997,94.809998,94.110001,94.150002,68.190575,5231500,CAT,0,2.266966,96.271975,88.886025,68.687627,82.330220,50.929600,90.157333,85.266833,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...


In [15]:
_d = data_features_cov1["cov_list"][0]
_d

NameError: name 'data_features_cov1' is not defined

In [20]:
# check_dir()

save_dataset(dataframe=data_features_cov1, type=FileType.JSON, filename)

# df_json = pd.read_json(filename, orient="split")
# df_json.head()

Saving data into: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-14T20-13-55.json


KeyboardInterrupt: 